<a href="https://colab.research.google.com/github/Anurag38/NLP-Practicals/blob/main/Practical%209%20-%20Text%20Processing/Text_Processing_Using_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from gensim.models import KeyedVectors
from gensim.models.wrappers import FastText
from gensim.models import FastText



In [2]:
import itertools
from gensim.models.word2vec import Text8Corpus
#from glove import Corpus, Glove
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
# from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
#from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
import gensim
from gensim.models.word2vec import Word2Vec
from gensim.models.doc2vec import TaggedDocument
import multiprocessing
from sklearn import utils
%matplotlib inline

In [3]:
# provide sql-like data manipulation tools. very handy.
import pandas as pd 
pd.options.mode.chained_assignment = None

# high dimensional vector computing library.
import numpy as np 
from copy import deepcopy
from string import punctuation
from random import shuffle

import gensim

# the word2vec model gensim class
from gensim.models.word2vec import Word2Vec 

# we'll talk about this down below
LabeledSentence = gensim.models.doc2vec.LabeledSentence 

from tqdm import tqdm
tqdm.pandas(desc="progress-bar")

# a tweet tokenizer from nltk.
from nltk.tokenize import TweetTokenizer 
tokenizer = TweetTokenizer()

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
train= pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/NLP/Practical 9/taska.txt',header=None,sep='\t',names=['id','text','class'])
#valid_df= pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/AMI/dataset/agr_en_dev.csv',header=None,names=['id','text','class'])

#test_df = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/AMI/dataset/agr_en_fb_gold.csv',header=None,names=['id','text','class'])


In [6]:
train.info()
train.head()
train['class'].value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13240 entries, 0 to 13239
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      13240 non-null  int64 
 1   text    13240 non-null  object
 2   class   13240 non-null  object
dtypes: int64(1), object(2)
memory usage: 310.4+ KB


NOT    8840
OFF    4400
Name: class, dtype: int64

In [7]:
import re
def clean_text(comment):
    comment = comment.strip().strip('"')
    comment = comment.replace('_', ' ')
    comment = comment.replace("\\\\", "\\")
    comment = comment.replace('\\n', ' ')
    comment = comment.replace('\\n', ' ')
    comment = comment.lower()
    comment = re.sub(r'@[A-Za-z0-9]+','',comment)
    
    comment = re.sub('https?://[A-Za-z0-9./]+','',comment)
    commment = re.sub("[^a-zA-Z]", " ", comment)
    comment = re.sub(r'[^\w\s]','',comment)

    
    return comment

train['text'] = train['text'].fillna("something").values
train['ctext']=train['text'].apply(clean_text)
#train_df['ctext']=train_df['text']
#valid_df['ctext']=valid_df['text'].apply(clean_text)
#test_df['ctext']=test_df['text'].apply(clean_text)
train.head()


,id,text,class,ctext
0,86426,@USER She should ask a few native Americans wh...,OFF,she should ask a few native americans what th...
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,go home youre drunk maga trump2020 url
2,16820,Amazon is investigating Chinese employees who ...,NOT,amazon is investigating chinese employees who ...
3,62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,someone shouldvetaken this piece of shit to a...
4,43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,obama wanted liberals amp illegals to move i...


In [8]:
#tr=train_df[train_df['text']=='something']
#tr.head()

In [9]:
#train_df['text'] = train_df['text'].fillna("something").values
train['class'] = train['class'].map({'OFF': 0, 'NOT': 1})


In [10]:
train_df, test_df = train_test_split(train,test_size = 0.1)


In [11]:
import nltk
nltk.download('punkt')
train_df['tokenized_sents'] = train_df.apply(lambda row: nltk.word_tokenize(row['ctext']), axis=1)
test_df['tokenized_sents'] = test_df.apply(lambda row: nltk.word_tokenize(row['ctext']), axis=1)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [12]:
train_df.head()

,id,text,class,ctext,tokenized_sents
10099,40731,@USER fuck yeah !,0,fuck yeah,"[fuck, yeah]"
6109,10009,@USER @USER @USER @USER Did you attend Governo...,0,did you attend governor abbotts meetings o...,"[did, you, attend, governor, abbotts, meetings..."
8411,85315,@USER @USER Sloppy and pathetic.,1,sloppy and pathetic,"[sloppy, and, pathetic]"
9048,38056,@USER @USER @USER @USER Read @USER that I just...,1,read that i just posted he got too close ...,"[read, that, i, just, posted, he, got, too, cl..."
8149,58969,@USER Real victims of sexual assault/harassmen...,1,real victims of sexual assaultharassment see ...,"[real, victims, of, sexual, assaultharassment,..."


In [13]:
x_train=train_df['tokenized_sents']
x_test=test_df['tokenized_sents']

y_train=train_df['class']
y_test=test_df['class']


In [14]:
def labelizeTweets(tweets, label_type):
    labelized = []
    for i,v in tqdm(enumerate(tweets)):
        label = '%s_%s'%(label_type,i)
        labelized.append(LabeledSentence(v, [label]))
    return labelized

x_train = labelizeTweets(x_train, 'TRAIN')

x_test = labelizeTweets(x_test, 'TEST')


0it [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).
  """
11916it [00:00, 164447.87it/s]
1324it [00:00, 7812.35it/s]


In [15]:
#tweet_w2v = FastText(size=400, min_count=1,sg=1)
tweet_w2v = Word2Vec(size=300, min_count=1,sg=1) #SKIPGRAM
tweet_w2v.build_vocab([x.words for x in tqdm(x_train)])
tweet_w2v.train([x.words for x in tqdm(x_train)],total_examples=len(x_train),epochs=10)


100%|██████████| 11916/11916 [00:00<00:00, 1659505.48it/s]


(1817689, 2333720)

In [16]:
tweet_w2v['hello']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([ 0.0245385 ,  0.01042343,  0.07029286, -0.08592399, -0.00728595,
        0.05229944, -0.08987518, -0.01976275,  0.05312984,  0.07454742,
       -0.00459528, -0.07464006,  0.13708974,  0.01855008, -0.04661439,
       -0.05090912, -0.15324254, -0.05939812,  0.13873217, -0.20823406,
        0.06541061, -0.10304102, -0.00307648,  0.01553386,  0.04710938,
       -0.04464864,  0.03309749, -0.04163473, -0.05369332,  0.07051885,
        0.01935233, -0.00962516,  0.00170115, -0.11199442, -0.14744046,
        0.02202365, -0.13418731, -0.02419685, -0.07234977,  0.03682282,
        0.07348078,  0.2031062 ,  0.0991015 , -0.01744791, -0.0072599 ,
       -0.20430917,  0.09818937,  0.11944608,  0.07149309,  0.16775118,
        0.0611512 , -0.09724732, -0.08685201, -0.08746489,  0.10104701,
        0.04837935, -0.01504948,  0.0680185 , -0.02765145,  0.08062874,
       -0.04698613, -0.03928098,  0.00120092, -0.10226735,  0.14784406,
        0.05963786, -0.08266125, -0.11659907,  0.05641096,  0.13

In [18]:
#tweet_w2v.save('off-sg.vec')
#tweet_w2v.save('fb-en-w2vec-cbow.bin')
#tweet_w2v.wv.save_word2vec_format('fb-hi-w2vec-cbow.txt')

In [19]:
print ('building tf-idf matrix ...')
vectorizer = TfidfVectorizer(min_df=1)
#matrix = vectorizer.fit_transform(x_train)
matrix = vectorizer.fit_transform(train_df['ctext'])

tfidf = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))
print ('vocab size :', len(tfidf))

building tf-idf matrix ...
vocab size : 19409


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [20]:
#tweet_w2v['messi']

In [21]:
#tweet_w2v=FastText.load('fb-en-fasttext-sg.vec')


In [22]:
def buildWordVector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens:
        try:
            vec += tweet_w2v[word].reshape((1, size)) * tfidf[word]
            count += 1.
        except KeyError: # handling the case where the token is not
                         # in the corpus. useful for testing.
            continue
    if count != 0:
        vec /= count
    return vec

In [23]:
#tweet_w2v=Word2Vec.load('D:/ps/model/English/fasttext/coling-fasttext-eng-cbow.bin')
#from gensim.models.wrappers import FastText

#tweet_w2v= FastText.load_fasttext_format('D:/ps/model/English/fasttext/coling-fasttext-eng-cbow')

n_dim=300
from sklearn.preprocessing import scale
train_vecs_w2v = np.concatenate([buildWordVector(z, n_dim) for z in tqdm(map(lambda x: x.words, x_train))])
train_vecs_w2v = scale(train_vecs_w2v)



test_vecs_w2v = np.concatenate([buildWordVector(z, n_dim) for z in tqdm(map(lambda x: x.words, x_test))])
test_vecs_w2v = scale(test_vecs_w2v)



0it [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
11916it [00:02, 3972.92it/s]
0it [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
1324it [00:00, 4925.95it/s]


In [24]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

encoder = LabelEncoder()
encoder.fit(y_train)
encoded_Y = encoder.transform(y_train)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)
dummy_y

#encoder = LabelEncoder()
#encoder.fit(y_valid)
#vencoded_Y = encoder.transform(y_valid)
# convert integers to dummy variables (i.e. one hot encoded)
#vdummy_y = np_utils.to_categorical(vencoded_Y)
#vdummy_y



encoder = LabelEncoder()
encoder.fit(y_test)
tencoded_Y = encoder.transform(y_test)
# convert integers to dummy variables (i.e. one hot encoded)
tdummy_y = np_utils.to_categorical(tencoded_Y)
tdummy_y

array([[0., 1.],
       [1., 0.],
       [0., 1.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [25]:
from keras.models import Sequential
model = Sequential()
model.add(Dense(7, activation='relu', input_dim=300))
#model.add(Dense(256, activation='relu', input_dim=300))
#model.add(Dense(256, activation='relu', input_dim=300))

model.add(Dense(2, activation='softmax'))
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_vecs_w2v,dummy_y, epochs=12, batch_size=8, verbose=2)#validation_data=(valid_vecs_w2v, vdummy_y),

Epoch 1/12
1490/1490 - 2s - loss: 0.6266 - accuracy: 0.6743 - 2s/epoch - 2ms/step
Epoch 2/12
1490/1490 - 2s - loss: 0.5875 - accuracy: 0.6957 - 2s/epoch - 1ms/step
Epoch 3/12
1490/1490 - 2s - loss: 0.5744 - accuracy: 0.7069 - 2s/epoch - 1ms/step
Epoch 4/12
1490/1490 - 1s - loss: 0.5663 - accuracy: 0.7103 - 1s/epoch - 1ms/step
Epoch 5/12
1490/1490 - 1s - loss: 0.5632 - accuracy: 0.7124 - 1s/epoch - 968us/step
Epoch 6/12
1490/1490 - 1s - loss: 0.5600 - accuracy: 0.7169 - 1s/epoch - 994us/step
Epoch 7/12
1490/1490 - 2s - loss: 0.5574 - accuracy: 0.7170 - 2s/epoch - 1ms/step
Epoch 8/12
1490/1490 - 1s - loss: 0.5542 - accuracy: 0.7234 - 1s/epoch - 986us/step
Epoch 9/12
1490/1490 - 1s - loss: 0.5540 - accuracy: 0.7219 - 1s/epoch - 985us/step
Epoch 10/12
1490/1490 - 2s - loss: 0.5525 - accuracy: 0.7260 - 2s/epoch - 1ms/step
Epoch 11/12
1490/1490 - 2s - loss: 0.5513 - accuracy: 0.7234 - 2s/epoch - 1ms/step
Epoch 12/12
1490/1490 - 1s - loss: 0.5491 - accuracy: 0.7270 - 1s/epoch - 995us/step


In [26]:
#score = model.evaluate(valid_vecs_w2v, vdummy_y, batch_size=8, verbose=2)
#print (score[1])

score = model.evaluate(test_vecs_w2v, tdummy_y, batch_size=8, verbose=2)
print (score[1])



166/166 - 0s - loss: 0.6074 - accuracy: 0.6896 - 307ms/epoch - 2ms/step
0.689577043056488
